In [1]:
# Imports
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

### Gold Commodity

In [2]:
# Reading Gold Returns
filepath_1 = Path("..\\Commodity_Data\\GC_close_prices.csv")
gold_df = pd.read_csv(filepath_1)
#change DateTime Format
gold_df['Date']= pd.to_datetime(gold_df['Date'])
gold_df['Date'] = gold_df['Date'].dt.strftime("%Y/%m/%d")
#Set Index
gold_df.set_index(pd.to_datetime(gold_df['Date'], infer_datetime_format=True), inplace=True)
gold_df.drop(columns=['Date'], inplace=True)
gold_df.rename(columns={'Close/Last': 'Gold_Close'},inplace = True)

In [3]:
gold_df.head()

,Gold_Close,Volume,Open,High,Low
Date,,,,,
2020-07-31,1985.9,275460,1973.6,2005.4,1971.4
2020-07-30,1966.8,268606,1986.4,1987.3,1952.3
2020-07-29,1976.7,191033,1971.9,1998.4,1955.7
2020-07-28,1963.9,154485,1963.2,2000.0,1927.5
2020-07-27,1955.4,127487,1929.0,1971.3,1926.8


In [4]:
#Calculate daily Gold returns
gold_returns = gold_df.copy()
gold_returns.drop(columns=['Open','High','Volume','Low'], inplace=True)
gold_returns = gold_returns.pct_change()
#Rename columns
gold_returns.rename(columns={'Gold_Close':'Gold_Rate%'},inplace = True)

### Silver Commodity

In [5]:
# Reading Silver returns
filepath_2 = Path("..\\Commodity_Data\\SI_close_prices.csv")
silver_df = pd.read_csv(filepath_2)
#Change DateTime Format
silver_df['Date']= pd.to_datetime(silver_df['Date'])
silver_df['Date']= silver_df['Date'].dt.strftime("%Y/%m/%d")
#Set Index
silver_df.set_index(pd.to_datetime(silver_df['Date'], infer_datetime_format=True), inplace=True)
silver_df.rename(columns={'Close/Last': 'Silver_Close'},inplace = True)
silver_df.drop(columns=['Date'], inplace=True)

In [6]:
#Calculate daily Silver returns
silver_returns = silver_df.copy()
silver_returns.drop(columns=['Open','High','Volume','Low'], inplace=True)
silver_returns = silver_returns.pct_change()
#Rename columns
silver_returns.rename(columns={'Silver_Close':'Silver_Rate%'},inplace = True)

### Daily Returns DF

In [7]:
#Concat Commodity DF
daily_commodity_df = pd.concat([silver_df,silver_returns,gold_df,gold_returns], axis="columns", join="inner")
daily_commodity_df.dropna(inplace = True)
daily_commodity_df.sort_index().tail()

,Silver_Close,Volume,Open,High,Low,Silver_Rate%,Gold_Close,Volume,Open,High,Low,Gold_Rate%
Date,,,,,,,,,,,,
2020-07-24,22.850,110586,22.835,23.185,22.580,-0.067385,1897.5,288088,1884.9,1904.6,1880.5,-0.029610
2020-07-27,24.501,183880,22.970,24.820,22.940,0.008272,1955.4,127487,1929.0,1971.3,1926.8,-0.004328
2020-07-28,24.300,250138,24.700,26.275,22.460,-0.000863,1963.9,154485,1963.2,2000.0,1927.5,-0.006475
2020-07-29,24.321,137360,24.580,25.120,23.435,0.041050,1976.7,191033,1971.9,1998.4,1955.7,0.005034
2020-07-30,23.362,151041,24.450,24.580,23.000,-0.035266,1966.8,268606,1986.4,1987.3,1952.3,-0.009618


In [8]:
#Write to CSV
daily_commodity_df.to_csv('daily_commodity_df.csv', encoding='utf-8', index=True)

In [9]:
# Set weights
#weights = [0.5, 0.5]
# Calculate portfolio weighted return
#daily_weighted_returns = daily_commodity_df.dot(weights)

### Monthly Returns DF

In [10]:
#Gold Monthly DF
gold_monthly_close = gold_df.resample('1M').mean()
#Silver Monthly DF
silver_monthly_close = silver_df.resample('1M').mean()

In [11]:
#Concat Monthly close prices
monthly_close = pd.concat([silver_monthly_close, gold_monthly_close], axis="columns", join="inner")
monthly_close.sort_index().head()

,Silver_Close,Open,High,Low,Gold_Close,Open,High,Low
Date,,,,,,,,
2015-01-31,17.208000,16.940000,17.320000,16.815000,1279.200000,1259.000000,1285.400000,1257.500000
2015-02-28,16.743895,16.803684,17.049211,16.572895,1225.484211,1230.905263,1238.789474,1219.878947
2015-03-31,16.215545,16.222182,16.423773,16.049227,1177.854545,1180.418182,1188.804545,1171.713636
2015-04-30,16.337714,16.351429,16.556190,16.131190,1199.828571,1199.595238,1207.947619,1191.247619
2015-05-31,16.818700,16.766750,17.001500,16.593500,1198.200000,1198.000000,1205.100000,1190.650000


In [12]:
#Calculate monthly returns
monthly_returns = monthly_close.copy()
monthly_returns.drop(columns=['Open','High','Low'], inplace=True)
monthly_returns = monthly_returns.pct_change()
monthly_returns.dropna(inplace = True)
monthly_returns.rename(columns={'Silver_Close':'Monthly_Silver_%','Gold_Close':'Monthly_Gold_%'},inplace = True)

In [13]:
# Set weights
#weights = [0.5, 0.5]
# Calculate portfolio weighted return
#monthly_weighted_returns = monthly_returns.dot(weights)

In [14]:
monthly_commodity_df = pd.concat([monthly_close, monthly_returns], axis="columns", join="inner")
monthly_commodity_df.head()

,Silver_Close,Open,High,Low,Gold_Close,Open,High,Low,Monthly_Silver_%,Monthly_Gold_%
Date,,,,,,,,,,
2015-02-28,16.743895,16.803684,17.049211,16.572895,1225.484211,1230.905263,1238.789474,1219.878947,-0.026970,-0.041992
2015-03-31,16.215545,16.222182,16.423773,16.049227,1177.854545,1180.418182,1188.804545,1171.713636,-0.031555,-0.038866
2015-04-30,16.337714,16.351429,16.556190,16.131190,1199.828571,1199.595238,1207.947619,1191.247619,0.007534,0.018656
2015-05-31,16.818700,16.766750,17.001500,16.593500,1198.200000,1198.000000,1205.100000,1190.650000,0.029440,-0.001357
2015-06-30,16.028045,16.100227,16.236591,15.908636,1181.159091,1182.981818,1188.672727,1175.100000,-0.047010,-0.014222


In [15]:
#Write to CSV
monthly_commodity_df.to_csv('monthly_commodity_df.csv', encoding='utf-8', index=True)